## Enterprise Q&A Chatbot template
This sample demonstrates how to create ChatGPT-like experiences over your own data using the Retrieval Augmented Generation pattern.

The template includes sample data so it's ready to try end to end. In this sample application we use a fictitious company, and the experience allows its employees to ask questions about the benefits, internal policies, as well as job descriptions and roles.

![](./flow_bot.png)

## Import open source dependencies (LangChain, FAISS)

In [1]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

from utils import get_apikey, get_embeddings, get_vectorstore, create_or_get_vectorstore, get_chat_history, get_prompt_context, chat
API_Key= get_apikey()
embeddings = get_embeddings(API_Key)

## Input parameters for DataSet and Vector Store

In [2]:
datasetpath = "/mnt/data/healthcareplandetails"
store_name = "policy_document_store"

## Input parameters for Responsible AI Safety to prevent harms/hallucinations

In [3]:
background = """You are an AI assistant for answering questions about information in 
                employee benefits and health care policy documentation"""

safety = """If the question is not about health care policy coverage or employee benefits, 
        politely inform them that you are tuned to only answer questions about health care policy coverage or employeee benefits"""

## Initialize Vector Store with Embeddings from Enterprise Data (eg PDFs and more)

In [4]:
# get store if exists
vectorstore=get_vectorstore(store_name)

if not vectorstore:
    #load all the PDFs and extract the text
    loader = DirectoryLoader(datasetpath)
    data = loader.load()
    
    # Chunk your data up into smaller documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(data)
    
    #Create embeddings of your documents to get ready for semantic search
    vectorstore = create_or_get_vectorstore(store_name, texts, embeddings)
    


## Initialize Chat with LangChain, LLM & Vector Store to enrich results 

In [5]:
QA_PROMPT, CONDENSE_QUESTION_PROMPT = get_prompt_context(background, safety)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
retrievalchain = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory, qa_prompt=QA_PROMPT,
                                                     condense_question_prompt=CONDENSE_QUESTION_PROMPT, get_chat_history=get_chat_history)

In [ ]:
chat(memory, retrievalchain)

Human:


 Does my policy cover annual hearing exams?


AI:

Yes, your policy does cover annual hearing exams. The Northwind Standard plan covers all types of hearing care services, including hearing tests, hearing aids and hearing aid accessories, as well as cochlear implants and bone-anchored hearing aids. Additionally, the plan covers routine aid maintenance and repair, as well as counseling and hearing aid evaluations. The plan also covers up to $1,500 every year for all hearing aid services, including the hearing aid itself.
Human:


 What about vision?


AI:


Yes, the Northwind Standard plan does cover vision services. The plan provides coverage for medical, vision, and dental services, as well as preventive care services and prescription drug coverage. However, it is important to note that the plan does not cover any services related to emergency care, mental health and substance abuse coverage, or out-of-network services. For more information, please refer to the plan documents.
Human:
